<a href="https://colab.research.google.com/github/DimaKav/DS-Unit-4-Sprint-2-NLP/blob/master/module2-Bag-of-Words/LS_DS_422_BOW_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import re
import string

!pip install -U nltk

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize # Sentence Tokenizer
from nltk.tokenize import word_tokenize # Word Tokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.probability import FreqDist

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Requirement already up-to-date: nltk in /usr/local/lib/python3.6/dist-packages (3.4.1)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 1) (optional) Scrape 100 Job Listings that contain the title "Data Scientist" from indeed.com

At a minimum your final dataframe of job listings should contain
- Job Title
- Job Description

In [1]:
# Code is a mod of: https://github.com/aakashtandel/Web-Scraping-Indeed/blob/master/Code/Project%203%20-%20Web%20Scraping%20Indeed%20Job%20Listings%20Jupyter%20Notebook.ipynb

# Get the html from the page and store in soup var
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd

URL = "https://www.indeed.com/jobs?q=data+scientist&l=Los+Angeles,+CA&jt=fulltime"
# Do a request on the url
page = requests.get(URL)
# Specify a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
soup = BeautifulSoup(page.text, "html.parser")

print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="en">
 <head>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <script src="/s/f2cb3a7/en_US.js" type="text/javascript">
  </script>
  <script>
   !function(n){function r(n){for(var r=a,t=n.length;t;)r=33*r^n.charCodeAt(--t);return r>>>0}var t=this['indeed.i18n.localeData'],e=t['']||{},a=e.salt;if(e.hasOwnProperty('salt'))for(var i in n)t[function(n){var t=r(n);return e.hasOwnProperty('id_length')&&(t=String(t).substring(0,e.id_length)),t}(i)]=n[i];else for(var i in n)t[i]=[null].concat(n[i])}({"indeedapply_serp_label":["Apply instantly"]});
  </script>
  <link href="/s/97464e7/jobsearch_all.css" rel="stylesheet" type="text/css"/>
  <link href="http://rss.indeed.com/rss?q=data+scientist&amp;l=Los+Angeles%2C+CA&amp;jt=fulltime" rel="alternate" title="Data Scientist Jobs, Employment in Los Angeles, CA" type="application/rss+xml"/>
  <link href="/m/jobs?q=data+scientist&amp;l=Los+Angeles%2C+CA&amp;jt=fulltime" media="only screen 

In [2]:
def parse(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
    df = pd.DataFrame(columns=["Title","Location","Company","Salary", "Synopsis"])
    for each in soup.find_all(class_= "result" ):
        try: 
            title = each.find(class_='jobtitle').text.replace('\n', '')
        except:
            title = 'None'
        try:
            location = each.find('span', {'class':"location" }).text.replace('\n', '')
        except:
            location = 'None'
        try: 
            company = each.find(class_='company').text.replace('\n', '')
        except:
            company = 'None'
        try:
            salary = each.find('span', {'class':'no-wrap'}).text
        except:
            salary = 'None'
        synopsis = each.find('div', {'class':'summary'}).text.replace('\n', '')
        df = df.append({'Title':title, 'Location':location, 'Company':company, 'Salary':salary, 'Synopsis':synopsis}, ignore_index=True)
    return df
  
df = parse(URL)
df

,Title,Location,Company,Salary,Synopsis
0,Scientist; Process Design and Vali...,None,International Pharmaceutical Products,None,Major Global leading Consumer Prod...
1,Data Scientist (Advisor),None,Southern California Edison,None,7+ years of experience with data a...
2,Senior Data Scientist - Clean Energy,None,Southern California Edison,None,With a proven track record of leve...
3,Data Scientist,None,Jobot,"\n $100,000 - $150,000 a year",3+ years’ experience in Machine Le...
4,Senior Data Scientist,None,SCAN Health Plan,None,"Leads the requirements gathering, ..."
5,Operations Research Analyst,None,"ExoAnalytic Solutions, Inc.",None,ExoAnalytic Solutions is looking f...
6,Data Scientist,"Santa Monica, CA",Retina AI,None,You will be creating algorithms th...
7,Data Scientist,"Los Angeles, CA",Lucky Day,None,Create data segmentation and model...
8,DATA SCIENTIST,"Los Angeles, CA 90032",California State University,"\n $5,325 - $9,354 a month",The incumbent will ensure data int...
9,Data Scientist/Visualization Master,"Los Angeles, CA 90067",Intermedia Advertising,None,Developing data analytics models t...


In [0]:
# Get more data

url_template1 = "http://www.indeed.com/jobs?q=data+scientist&l={}&start={}"
url_template2 = "https://www.indeed.com/jobs?q=title%3A%28data+scientist%29&l={}&sort=date&radius=25&start={}"
max_results_per_city = 1000 # Set this to a high-value (5000) to generate more results. 
# Crawling more results, will also take much longer. First test your code on a small number of results and then expand.
i = 0
results = []
df_more = pd.DataFrame(columns=["Title","Location","Company","Salary", "Summary"])
for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
    'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
    'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'Naples', 
    'Charlottesville', 'Richmond', 'Baltimore', 'Harrisonburg', 'San+Antonio', 'San+Diego', 'San+Jose'
    'Austin', 'Jacksonville', 'Indianapolis', 'Columbus', 'Fort+Worth', 'Charlotte', 'Detroit', 'El+Paso', 
    'Memphis', 'Boston', 'Nashville', 'Louisville', 'Milwaukee', 'Las+Vegas', 'Albuquerque', 'Tucson', 
    'Fresno', 'Sacramento', 'Long+Beach', 'Mesa', 'Virginia+Beach', 'Norfolk', 'Atlanta', 'Colorado+Springs',
    'Raleigh', 'Omaha', 'Oakland', 'Tulsa', 'Minneapolis', 'Cleveland', 'Wichita', 'Arlington', 'New+Orleans', 
    'Bakersfield', 'Tampa', 'Honolulu', 'Anaheim', 'Aurora', 'Santa+Ana', 'Riverside', 'Corpus+Christi', 'Pittsburgh', 
    'Lexington', 'Anchorage', 'Cincinnati', 'Baton+Rouge', 'Chesapeake', 'Alexandria', 'Fairfax', 'Herndon',
    'Reston', 'Roanoke']):
    for start in range(0, max_results_per_city, 10):
        # Grab the results from the request (as above)
        url = url_template2.format(city, start)
        # Append to the full set of results
        html = requests.get(url)
        soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
        for each in soup.find_all(class_= "result" ):
            try: 
                title = each.find(class_='jobtitle').text.replace('\n', '')
            except:
                title = None
            try:
                location = each.find('span', {'class':"location" }).text.replace('\n', '')
            except:
                location = None
            try: 
                company = each.find(class_='company').text.replace('\n', '')
            except:
                company = None
            try:
                salary = each.find('span', {'class':'no-wrap'}).text
            except:
                salary = None
            try:
                summary = each.find('div', {'class':'summary'}).text.replace('\n', '')
            except:
                summary = None
            df_more = df_more.append({'Title':title, 'Location':location, 'Company':company, 'Salary':salary, 'Summary':summary}, ignore_index=True)
            i += 1
            if i % 1000 == 0:
                print('You have ' + str(i) + ' results. ' + str(df_more.dropna().drop_duplicates().shape[0]) + " of these aren't rubbish.")

You have 1000 results. 35 of these aren't rubbish.
You have 2000 results. 59 of these aren't rubbish.
You have 3000 results. 80 of these aren't rubbish.
You have 4000 results. 110 of these aren't rubbish.


In [31]:
df_more.shape

(3802, 5)

In [32]:
df_more = df_more.drop_duplicates()

(2293, 5)

In [60]:
df_more.sample(5)

,Title,Location,Company,Salary,Summary
2856,Data Scientist,None,Disney Streaming Services,None,3+ years of experience as a Data S...
1213,Data Scientist,"Baton Rouge, LA 70802",Romph & Pou Agency,"\n $107,000 - $157,000 a year (...",Responsibilities for Data Scientis...
1475,Data Scientist - Data Scientist Senior (Market...,"Columbus, OH 43215 (Downtown area)",American Electric Power,"\n $92,000 - $136,000 a year (I...",Data Scientist Senior:. Apply data visualizati...
2427,"Data Scientist, Product analytics",None,eBay Inc.,None,"Data Scientist, Product Analytics. We are look..."
3581,Senior Data Scientist,None,Ascensus,None,Perform complex data analysis; Bui...


In [0]:
df_more.to_csv('data_scientist_jobs_indeed.csv', encoding='utf-8', index=False)

In [0]:
# df_more = pd.read_csv('indeed_data_science_unclean.csv')
# del df_more['Unnamed: 0']
# del df_more['Salary']
# df_more.rename(columns={'Synopsis':'Summary'}, inplace=True)

In [0]:
df_more.sample(5)

,Title,Location,Company,Summary
2580,Cloud Solutions Architect,NaN,Pythian,Prior experience working as/with M...
8200,Chemical Engineer- Relocation to S...,NaN,Saudi Aramco,Petroleum engineers and earth scie...
5048,Data Analyst,"Los Angeles, CA 90036","Research & Development Institute, Inc.",You will reports to Clinical Data ...
7899,Data Scientist,"Austin, TX",Cloudflare,Proven track record of applying da...
6850,Lead Data Scientist,"Pittsburgh, PA 15222 (Strip District area)","Federated Investors, Inc.",Leverage and synthesize large volu...


In [0]:
df_more.isna().sum()

Title          0
Location    3426
Company        5
Summary        0
dtype: int64

## 2) Use NLTK to tokenize / clean the listings 

In [0]:
import spacy
nlp = spacy.load('en', parser=False, entity=False)        

df_more['Summary_tokens'] = df_more['Summary'].apply(lambda x: nlp.tokenizer(x))

In [0]:
df_more.head()

,Title,Location,Company,Summary,Summary_tokens
0,Cloud Solutions Architect,NaN,Pythian,Prior experience working as/with M...,"( , Prior, experience, working, as,..."
1,Senior Process & Control System Te...,NaN,Saudi Aramco,Petroleum engineers and earth scie...,"( , Petroleum, engineers, and, eart..."
2,Business System Analyst- Relocatio...,NaN,Saudi Aramco,Petroleum engineers and earth scie...,"( , Petroleum, engineers, and, eart..."
3,GIS Intern,"Anchorage, AK 99503 (Midtown area)",HDR,We create an unshakable foundation...,"( , We, create, an, unshakable, fou..."
4,Project Scientist,"Anchorage, AK 99501 (Fairview area)",Weston Solutions,The Project Scientist carries out ...,"( , The, Project, Scientist, carrie..."


In [0]:
# New stop words list 
customize_stop_words = [
    'as','and','we',
    'data scientist'
]

# Mark them as stop words
for w in customize_stop_words:
    nlp.vocab[w].is_stop = True


# Convert each row into spacy document and return the lemma of the tokens in 
# the document if it is not a sotp word. Finally join the lemmas into as a string
df_more['Summar_lema'] = df_more['Summary'].apply(lambda text: 
                                          " ".join(token.lemma_ for token in nlp(text) 
                                                   if not token.is_stop))

df_more.head()

In [0]:
df_more.to_csv('indeed_salaries_tokenized_lematized.csv')

# 3) Use Scikit-Learn's CountVectorizer to get word counts for each listing.

In [0]:
df_more = pd.read_csv('indeed_salaries_tokenized_lematized.csv')

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text 

# Get rid of numbers in summary
df_more['Summary'] = df_more['Summary'].str.replace('\d+', '')

# Customize CountVectorizer stopwords
my_additional_stop_words = ['youtube','yr','yt','aa','_must','yrs','zone','data',
                            'scientists','learnings','engineers','machine',
                            'learning','scientist', "science"]
stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)

# Instantiate CountVectorizer
cvec = CountVectorizer(stop_words=stop_words)

# Fir the CountVectorizer and transform the data
vectorizers = cvec.fit_transform(df_more['Summary'].values)


df_vec  = pd.DataFrame(vectorizers.todense(), columns=cvec.get_feature_names())
print (df_vec.shape)
df_vec.head()


(10020, 5841)


,_epic,aacr,ab,abbott,abilities,abilitiesproven,ability,able,abreast,absci,...,years,yes,yield,yielding,york,young,zestfinance,zillow,zin,zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 4) Visualize the most common word counts

In [0]:
df_vec.sum().sort_values().tail(20)

mining            636
visualization     643
clinical          698
technical         700
statistical       704
working           708
health            782
support           782
years             823
business          840
scientific        881
manage            914
project           930
including         977
research         1026
analysis         1065
team             1489
work             1552
analytics        2020
experience       2022
dtype: int64

 # 5) Use Scikit-Learn's tfidfVectorizer to get a TF-IDF feature matrix

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x = v.fit_transform(df_more['Summary'])

x.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Stretch Goals

* Try different visualizations for words and frequencies - what story do you want to tell with the data?
* Scrape Job Listings for the job title "Data Analyst". How do these differ from Data Scientist Job Listings
* Try and identify requirements for experience specific technologies that are asked for in the job listings. How are those distributed among the job listings?
Use a clustering algorithm to cluster documents by their most important terms. Do the clusters reveal any common themes?

Hint: K-means might not be the best algorithm for this. Do a little bit of research to see what might be good for this. Also, remember that algorithms that depend on Euclidean distance break down with high dimensional data.